In [1]:
# Notebook to generate the dataframe with the daily range-trail to use for market operation

# 1. First calculate the daily range (max - min)
# 2. Calculate the average for the last years
# 3. Adjust the average range with a percentage
# 4. Round the final number to the next integer

In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
df_es=pd.read_csv('../Data/ES_00.csv',dtype={'date': str,'time': str,'open': float ,'max' : float ,'min' : float ,'close' : float ,'volume' : int,
                                              'datetime_US' : str, 'datetime_EU' : str, }, parse_dates=['datetime_US', 'datetime_EU'])
df_es.head()

,date,time,open,max,min,close,volume,datetime_US,datetime_EU
0,09/27/2009,18:15,1043.75,1044.00,1043.50,1043.75,631,2009-09-27 18:15:00,2009-09-28 00:15:00
1,09/27/2009,18:30,1043.75,1044.00,1043.50,1043.50,177,2009-09-27 18:30:00,2009-09-28 00:30:00
2,09/27/2009,18:45,1043.75,1043.75,1043.50,1043.75,180,2009-09-27 18:45:00,2009-09-28 00:45:00
3,09/27/2009,19:00,1043.75,1043.75,1040.25,1041.50,3017,2009-09-27 19:00:00,2009-09-28 01:00:00
4,09/27/2009,19:15,1041.50,1041.50,1040.00,1040.75,1232,2009-09-27 19:15:00,2009-09-28 01:15:00


In [6]:
df_es.iloc[-1,:]['datetime_EU'].year

2019

In [8]:
df_es.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241774 entries, 0 to 241773
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         241774 non-null  object        
 1   time         241774 non-null  object        
 2   open         241774 non-null  float64       
 3   max          241774 non-null  float64       
 4   min          241774 non-null  float64       
 5   close        241774 non-null  float64       
 6   volume       241774 non-null  int64         
 7   datetime_US  241774 non-null  datetime64[ns]
 8   datetime_EU  241774 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 16.6+ MB


In [3]:
## function to transform an integer with format YYYYMMDD into a datatype datetime format

def int2date(argdate: int) -> date:
    """
    If you have date as an integer, use this method to obtain a datetime.date object.
    -Parameters
    ----------
    argdate : int
      Date as a regular integer value (example: 20160618)

    -Returns
    -------
    dateandtime.date
      A date object which corresponds to the given value `argdate`.
    """
    year = int(argdate / 10000)
    month = int((argdate % 10000) / 100)
    day = int(argdate % 100)

    return date(year, month, day)



In [4]:

## function to go throughout the dataframe and calculate the daily range

def daily_range (data_rd, data_wr, timetable_ini, timetable_end):
    """
    This function create a dataframe with maximum minimum and range for a daily base in a especific timetable
    -Parameters
    ----------
    data_rd : dataframe with original data (intraday movements of a future contract) ['date', 'time', 'open', 'max', 'min', 'close', 'volume']
    data_wr : new dataframe to create with columns ['date', 'timetable', 'max', 'min', 'range']. It must be empty 
    timetable_ini : time the operation start everyday. Example='09:00'
    timetable_end : time the operation finish everyday. Example='18:00' 

    -Returns
    -------
    nothing
    The function fill the dataframe data_wr with daily maximum, minimum, range(maximum-minimum)
    """
#    data_wr=pd.DataFrame(columns=['date', 'timetable', 'max', 'min', 'range'])
#    data_wr['date']=pd.to_datetime(data_wr['date'], format='%Y-%m-%d')
    
    hour_ini=int(timetable_ini[0:2])                                  # initial values
    minu_ini=int(timetable_ini[3:])
    hour_end=int(timetable_end[0:2])
    minu_end=int(timetable_end[3:])
    tmtb_ini = hour_ini * 100 + minu_ini                              # integer value
    tmtb_end = hour_end * 100 + minu_end                              # integer value
    minim = 0.0
    maxim = 999999.9
    flag_in = False
    my_dt_ant =20000101
    j=0
#    print(tmtb_ini, tmtb_end)
    for i in range(len(data_rd)):
        my_dt=data_rd['datetime_EU'][i].year * 10000 + data_rd['datetime_EU'][i].month *100 + data_rd['datetime_EU'][i].day
        my_ho=data_rd['datetime_EU'][i].hour
        my_mi=data_rd['datetime_EU'][i].minute
        my_tmtb = my_ho * 100 + my_mi                               # timetable as integer
#        print(my_dt, my_tmtb)
        if (my_dt_ant == my_dt):
            if ((my_tmtb == tmtb_ini) & (flag_in == False) ):      # the start of timetable 
                maxim=data_rd['max'][i]
                minim=data_rd['min'][i]
                flag_in = True            
            elif ((my_tmtb > tmtb_ini) & (my_tmtb < tmtb_end) & (flag_in == True) ): # update max and min values
                if (data_rd['max'][i]>maxim):
                    maxim=data_rd['max'][i]
                if (data_rd['min'][i]<minim):
                    minim=data_rd['min'][i]
            elif ((my_tmtb == tmtb_end) & (flag_in == True) ):    # the end of timetable and save daily values
                flag_in=False
                my_dt_ant_date=int2date(my_dt_ant)
                data_wr.loc[j]=[my_dt_ant_date, timetable_ini+'-'+timetable_end, maxim, minim, maxim-minim]
                j+=1
                maxim=99999.9
                minim=0.0
#            print(i, len(data_rd), maxim, minim)
        my_dt_ant=my_dt
    data_wr['date']=pd.to_datetime(data_wr['date'], format='%Y-%m-%d')



In [2]:
# function to locate the first date of a period (year or month) in a dataframe with a column named 'date'

def locate_1st_day (my_period, my_data, my_df, my_column_date):
    """
    This function locate the first date of a period (year or month) in a dataframe with a column named 'date' with datetime datatype
    -Parameters
    ----------
    my_period : year 'Y', month 'M', day 'D'
    my_data : is an integer with the information: year with format YYYY and should be greater than 2009
                                                  month with format YYYYMM and should be greater than 200900
    my_df : this is the dataframe with a column with datetime datatype
    my_column_date : this is the name of the column with datetime datatype and we use to locate a date (1st day of a period)
    -Returns
    -------
    An integer with the index position in the dataframe
    Any error return -1
    """
    if (my_column_date not in my_df.columns):
        print('error column date does not exist')                                #print
        return -1
    if (my_column_date not in my_df.select_dtypes(['datetime64']).columns):
        print('error column date is not datetime datatype')                                #print
        return -1
    if ((my_period=='Y') & (isinstance(my_data, int)) & (my_data>2009) & (my_data<2050)):
        date_to_find = my_data * 10000 + 100 + 1
    elif ((my_period=='M') & (isinstance(my_data, int)) & (my_data>201000) & (my_data<205000)):
        date_to_find = my_data * 100 + 1
    elif ((my_period=='D') & (isinstance(my_data, int)) & (my_data>20100100) & (my_data<20500100)):
        date_to_find = my_data
    else:
        print(f'1st error input or argument {my_period} {my_data} {isinstance(my_data, np.int64)} '
              f'{isinstance(my_data, int)} {my_column_date}')                                          #print
        return -1
    
    for reg in range(len(my_df)):
        my_int_date=my_df[my_column_date][reg].year * 10000 + my_df[my_column_date][reg].month * 100 + my_df[my_column_date][reg].day
        if (my_int_date == date_to_find):
#            print(reg, my_df[my_column_date][reg])                                      #print
            return reg
        elif (my_int_date > date_to_find):
#            print(reg, my_df[my_column_date][reg])                                      #print
            return reg
    print(f'the data does not exist : {my_int_date}')                                             #print
    return -1

#locate_1st_day('D', 20150101, df_es, 'datetime_EU')


In [6]:
# function to locate the first date of a period (year or month) in a dataframe with a column named 'date'

def locate_lst_day (my_period, my_data, my_df, my_column_date):
    """
    This function locate the last date of a period (year or month) in a dataframe with a column named 'date' with datetime datatype
    -Parameters
    ----------
    my_period : year 'Y', month 'M'
    my_data : is an integer with the information: year with format YYYY and should be greater than 2009
                                                  month with format YYYYMM and should be greater than 200900
    my_df : this is the dataframe with a column called 'date' with datetime datatype
    my_column_date : this is the name of the column with datetime datatype and we use to locate a date (1st day of a period)
    -Returns
    -------
    An integer with the index position in the dataframe
    Any error return -1
    """
    if (my_column_date not in my_df.columns):
        print('error column date does not exist')                                #print
        return -1
    if (my_column_date not in my_df.select_dtypes(['datetime64']).columns):
        print('error column date is not datetime datatype')                                #print
        return -1
    if ((my_period=='Y') & (isinstance(my_data, int)) & (my_data>2009) & (my_data<2050)):
        date_to_find = my_data * 10000 + 100 + 1
    elif ((my_period=='M') & (isinstance(my_data, int)) & (my_data>200900) & (my_data<205000)):
        date_to_find = my_data * 100 + 1
    else:
        print('error input or argument')                                                 #print
        return -1
    in_period=False
    for reg in range(len(my_df)):
        my_int_date=my_df[my_column_date][reg].year * 10000 + my_df[my_column_date][reg].month * 100 + my_df[my_column_date][reg].day
        if ((my_period=='Y') & (int(my_int_date/10000) >= my_data)):
            if ((in_period==True) & (int(my_int_date/10000) > my_data)):
#                print(reg-1, my_df[my_column_date][reg-1], int(my_int_date/10000))                               #print
#                print(reg, my_df[my_column_date][reg], int(my_int_date/10000))
                return reg-1
            in_period=True
        elif ((my_period=='M') & (int(my_int_date/100) >= my_data)):
            if ((in_period==True) & (int(my_int_date/100) > my_data)):
#                print(reg-1, my_df[my_column_date][reg-1], int(my_int_date/100))                              #print
                return reg-1
            in_period=True
    print(f'the data does not exist : {my_int_date}')                                                                 #print
    if (in_period==True):
        return reg
    return -1

#locate_lst_day('Y', 2016, df_es, 'datetime_EU')

In [7]:
## function to go throughout the dataframe and calculate the daily range

def range_trail_estimation (years, adjustment, updating, my_df):
    """
    This function estimate and create the range-trail to use in the daily operation in the market according to the following parameters:
    Then the data frame df_es_day will be incremented with a new column 'trail' with the daily value o estimated range-trail 
    -Parameters
    ----------
    years : the quantity of years we are going to use to calculate the average. For example 4 years means that we will take the last 4 years to
            calculate the average range. 
    adjustment : the perdentage (%) we will apply to adjust the average range calculated. For example 0.70 (70%).
    updating : this is the time we are going to update the range-trail estimation. For example 'Y' means every year. From 1st of January 
               we will use the range-trail estimated until the 31th of December.
    
              formula >> range_trail = (average of range of last 4 years) * (adjustment)  
                      >> then round up to the closer integer, tenth, hundredth, thousandth according to the contract value
    my_df : this is the dataframe to work with and its columns ['date', 'timetable', 'max', 'min', 'range']
            we will create a new column range-trail
    -Returns
    -------
    nothing
    The function fill the dataframe df_es_day with daily range-trail
    """
    pd.options.mode.chained_assignment = None                                   # default='warn'   This is to avoid warnings of chained-assignment
    
    if ((years != 3) & (years != 4)):
        print('Error: Only 3 or 4 years')
        return -1
    if ((adjustment < 0.70) & (adjustment > 0.75)):
        print('Error: only 0.70 or 0.75 value for adjustment')
        return -1
    if (updating != 'Y'):
        print('Error: only Y for updating')
        return -1
    
    my_df['range-trail']=np.zeros(len(my_df))
    my_df['range-avg']=np.zeros(len(my_df))
    #my_df = my_df.reindex(columns = ['date', 'timetable', 'max', 'min', 'range', 'range-trail']) 
    
    first_year=df_es.iloc[0,:]['date'].year
    last_year=df_es.iloc[-1,:]['date'].year
    year_in = first_year + years + 1                          # 2009 + 4 + 1 = 2014
    year_out = last_year                                      # 2019
    #year_in = 2015                                   # GLOBAL VARIABLE ????????????????????????????????????????
    #year_out = 2019                                  # GLOBAL VARIABLE ???????????????????????????????????????
    
    total_loops = year_out - year_in +1                                    # 2019 - 2014 = 5 + 1
    loop_num = 0 
    
    while (loop_num <(total_loops)):
        
        year_to_start = year_in + loop_num - years                                      # 2014 + loop - 4 = 2010
        year_to_finish = year_out                                         # 2019
    
        pta_ini=locate_1st_day('Y', year_to_start, my_df, 'date')                   # 2010
        pta_end=locate_lst_day('Y', year_to_start+years-1, my_df, 'date')          # 2010 + 4 -1 = 2014
        my_range_sum=0.0
#        print(year_to_start, pta_ini, pta_end)
        for i in range(pta_ini,pta_end+1):
            my_range_sum += my_df['range'][i]
    
        my_range_avg = my_range_sum/(pta_end-pta_ini+1)
        my_range_trl = round(my_range_avg * adjustment, 0) + 1        # round up to nex integer > Becareful because can change with every contract!
    
        ptt_ini=locate_1st_day('Y', year_to_start + years, my_df, 'date')
        ptt_end=locate_lst_day('Y', year_to_start + years, my_df, 'date')
#        print(year_to_start+years, ptt_ini, ptt_end)
        for i in range(ptt_ini,ptt_end+1):
            my_df['range-trail'][i]=my_range_trl
            my_df['range-avg'][i]=my_range_avg

        loop_num +=1
        

In [8]:
# herewith is the workflow to follow

df_es_day=pd.DataFrame(columns=['date', 'timetable', 'max', 'min', 'range'])
daily_range (df_es, df_es_day, '09:00', '18:30')
range_trail_estimation (4, 0.70, 'Y', df_es_day)

the data does not exist : 20190906


In [9]:

df_es_day

,date,timetable,max,min,range,range-trail,range-avg
0,2009-09-28,09:00-18:30,1060.25,1037.00,23.25,0.0,0.000000
1,2009-09-29,09:00-18:30,1065.75,1053.25,12.50,0.0,0.000000
2,2009-09-30,09:00-18:30,1063.25,1041.50,21.75,0.0,0.000000
3,2009-10-01,09:00-18:30,1055.25,1031.50,23.75,0.0,0.000000
4,2009-10-02,09:00-18:30,1027.00,1012.00,15.00,0.0,0.000000
...,...,...,...,...,...,...,...
2554,2019-09-02,09:00-18:30,2926.50,2889.00,37.50,14.0,19.223518
2555,2019-09-03,09:00-18:30,2914.50,2891.00,23.50,14.0,19.223518
2556,2019-09-04,09:00-18:30,2936.25,2921.50,14.75,14.0,19.223518
2557,2019-09-05,09:00-18:30,2986.50,2950.50,36.00,14.0,19.223518


In [11]:
df_es_day[df_es_day['range-trail'] != 0]['date'].count()

1207

In [16]:
# Export a file to test

#filter=(df_es_day['date'] > '2018-01-01') & (df_es_day['date'] < '2018-12-31')
#df_es_day[filter].to_csv('test02.csv', index=False)


In [17]:
# Export a file to follow
df_es_day.to_csv('./Data/ES_02.csv',index=False)
df_es_day

,date,timetable,max,min,range,range-trail,range-avg
0,2009-09-28,09:00-18:30,1060.25,1037.00,23.25,0.0,0.000000
1,2009-09-29,09:00-18:30,1065.75,1053.25,12.50,0.0,0.000000
2,2009-09-30,09:00-18:30,1063.25,1041.50,21.75,0.0,0.000000
3,2009-10-01,09:00-18:30,1055.25,1031.50,23.75,0.0,0.000000
4,2009-10-02,09:00-18:30,1027.00,1012.00,15.00,0.0,0.000000
...,...,...,...,...,...,...,...
2554,2019-09-02,09:00-18:30,2926.50,2889.00,37.50,14.0,19.223518
2555,2019-09-03,09:00-18:30,2914.50,2891.00,23.50,14.0,19.223518
2556,2019-09-04,09:00-18:30,2936.25,2921.50,14.75,14.0,19.223518
2557,2019-09-05,09:00-18:30,2986.50,2950.50,36.00,14.0,19.223518


In [35]:
df_es_day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2559 entries, 0 to 2558
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2559 non-null   datetime64[ns]
 1   timetable    2559 non-null   object        
 2   max          2559 non-null   float64       
 3   min          2559 non-null   float64       
 4   range        2559 non-null   float64       
 5   range-trail  2559 non-null   float64       
 6   range-avg    2559 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 239.9+ KB


In [11]:
Filter=(df_es_day['date'] > '2015-01-01') & (df_es_day['date'] < '2019-12-31')
(df_es_day[Filter].groupby(['range-avg', pd.Grouper(freq='Y', key='date')])
             ['range-avg']
             .mean()
             .unstack(fill_value='-'))

date,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31
range-avg,,,,,
14.710732,14.7107,-,-,-,-
15.162768,-,15.1628,-,-,-
16.220117,-,-,-,16.2201,-
16.338278,-,-,16.3383,-,-
19.223518,-,-,-,-,19.2235


In [12]:
Filter=(df_es_day['date'] > '2015-01-01') & (df_es_day['date'] < '2019-12-31')
(df_es_day[Filter].groupby(['range-trail', pd.Grouper(freq='Y', key='date')])
             ['range-trail']
             .mean()
             .unstack(fill_value='-'))

date,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-12-31
range-trail,,,,,
11.0,11,-,-,-,-
12.0,-,12,12,12,-
14.0,-,-,-,-,14


In [26]:
int_date_01 = locate_1st_day('Y', 2015, df_es_day, 'date')
int_date_11 = locate_lst_day('Y', 2019, df_es_day, 'date')
d1 = df_es_day['date'][int_date_11]                               
d0 = df_es_day['date'][int_date_01]                               
delta = d1 - d0
total_delta=delta.total_seconds()
delta_years = total_delta/(365*24*60*60)  
print(int_date_01,int_date_11,d1,d0, delta, total_delta, delta_years)

the data does not exist : 20190906
1352 2558 2019-09-06 00:00:00 2015-01-02 00:00:00 1708 days 00:00:00 147571200.0 4.67945205479452
